# Поиск подходящих элементов программы для спортсменов

**Описание проекта**

Сервис  “Мой Чемпион” помогает спортивным школам фигурного катания,
<br>тренерам мониторить результаты своих подопечных и планировать дальнейшее развитие спортсменов.

**Цель**

Создать модель, помогающую находить элементы, которые могут быть успешно исполнены спортсменом на соревновании. 
<br>Сервис будет прогнозировать прогресс и возможное выполнение тех или иных элементов программы
<br>по истории предыдущих выступлений и выполнения элементов на соревнованиях.

**План выполнения**

- Объединение данных – спортсмены, юниты, соревнования, сегменты, школы, тренеры;
- Проведение анализа данных;
- Выделение всех элементов фигурного катания, представленых в виде объединенной строки с набором элементов, например `2F!<<+2Lo<<CCoSp2V*`;
  - Актуальный список элементов можно посмотреть [здесь](https://eislauf-union.de/files/users/997/Elemente-Liste2023_24.pdf) - категория `Single skating`, а также специальные отметки `Special codes` в конце документа;
- Выделение элементов успешно выполненных и с погрешностями (имеют специальные пометки);
- Агрегация датасета и подготовка обучающих признаков;
- Создание модели для предсказания вероятности успешного выполнения элементов;
- Улучшение решения: анализ признаков, выявление ошибок и слабых сторон модели;
- Создание функции (класса) для применения в сервисе:
  - возможность регулировки длины истории спортсмена для построения прогноза,
  - добавление запланированной программы элементов в качестве признака;
- Проведение тестирования решения;
- Создание документации;
- Заключение (отчёт по проделанной работе).

## Setting up the work environment

In [15]:
# !pip install pandas
# !pip install python-dotenv
# !pip install seaborn
# !pip install psycopg2-binary
!pip freeze > requirements.txt

In [12]:
import os
import random
import numpy as np
import pandas as pd
import seaborn as sns

from dotenv import load_dotenv
# os.environ['VIRTUAL_ENV']

Declaring Global Variables

In [16]:
ASSETS_DIR = 'assets'
RANDOM_STATE = 42

random.seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)
TRACKING_SERVER_PORT = 5000
TRACKING_SERVER_HOST = "127.0.0.1"
EXPERIMENT_NAME = 'goprotect_athletes'

load_dotenv()
sns.set_style("white")
sns.set_theme(style="whitegrid")
pd.options.display.max_rows = 20
pd.options.display.max_columns = 30

connection = {"sslmode": "require",
              "target_session_attrs": "read-write"}
postgres_credentials = {"host": os.getenv("DB_HOST"),
                        "port": os.getenv("DB_PORT"),
                        "dbname": os.getenv("DB_NAME"),
                        "user": os.getenv("DB_USERNAME"),
                        "password": os.getenv("DB_PASSWORD")}

Importing a Research Class

In [17]:
%load_ext autoreload
%autoreload 2

from research_class import DatasetExplorer

## EDA